In [478]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [479]:
import graphviz

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges

def draw_dot(root):
    dot = graphviz.Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # LR = left to right
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name=uid + n._op, label=n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot

In [480]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
            
        out._backward = _backward
        return out
    
        
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad  += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad

        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __truediv__(self, other):
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def tahn(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tahn')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [597]:
class Module:
    def reset_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):
    def __init__(self, number_of_inputs):
        self.w = [Value(random.uniform(-1,1)) for _ in range(number_of_inputs)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tahn()
        return out
    def parameters(self):
        return self.w + [self.b]

class Layer(Module):
    def __init__(self, number_of_inputs, number_of_outputs):
        self.neurons = [Neuron(number_of_inputs) for _ in range(number_of_outputs)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP(Module):
    def __init__(self, number_of_inputs, number_of_outputs):
        sz = [number_of_inputs] + number_of_outputs
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(number_of_outputs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [598]:
x = [2, 3,1]
n = MLP(3, [4,4,1])

In [599]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

In [600]:
for k in range(10):
    # forward pass
    predictions = [n(x) for x in xs]
    loss = sum((yout - ygt)**2  for ygt, yout in zip(ys, predictions))
    pred_data = [x.data for x in predictions]

    print(pred_data, loss.data)
    
    n.reset_grad()
    # back pass
    loss.backward()

    # patch parameters (weights and biases)
    for p in n.parameters():
        p.data += -0.1 * p.grad

[0.8461286966455079, 0.5300623145254527, 0.903714241636254, 0.8421053690773236] 6.013825692609902
[0.31908455498333355, -0.41587641092664385, 0.5056364479966639, 0.2945580112652556] 3.5694357235802254
[0.27509939466844036, -0.5717435979394359, 0.20555355548246074, 0.2490440251688843] 2.726178684786875
[0.3618711551033011, -0.6495332572550928, 0.036196616566118514, 0.34085840595697964] 2.0382064296400655
[0.34549245953805485, -0.7502697879220012, -0.4246142537886737, 0.3507478810554078] 1.2433423702432749
[0.821791214055427, -0.6523731422573289, 0.005757775306581851, 0.820372071238309] 1.196417698992742
[-0.3379883569621506, -0.9162565757264602, -0.9066160909133871, -0.014732200922720652] 2.835627798541099
[0.9829364557636444, -0.09890790194548756, 0.9725694218850612, 0.9831551924738731] 4.703572005414746
[0.9550441832405484, -0.8334944870341603, 0.8611625658176346, 0.9555359716747966] 3.4956482575243126
[0.9031843279855134, -0.8820803029046406, -0.21175734989561507, 0.9039787576275136]